#### 1) 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os
from pathlib import Path

# rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'

rating_file_path = Path.joinpath(Path.cwd(),'data','ml-1m','ratings.dat')

ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings_over3 = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings_over3)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'rating':'counts'}, inplace=True)

In [4]:
ratings = ratings.dropna()

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000204    1
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 1000209, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
# movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
movie_file_path = Path.joinpath(Path.cwd(),'data','ml-1m','movies.dat')

cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

In [7]:
movies['title'] = movies['title'].str.lower() # 검색을 쉽게 하기 위해 영화제목 문자열을 소문자로 바꿔줍시다.
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


#### 2) 분석해 봅시다.  

ratings에 있는 유니크한 영화 개수

In [8]:
ratings['movie_id'].nunique()

3706

In [9]:
ratings['movie_id']

0          1193
1           661
2           914
3          3408
4          2355
           ... 
1000204    1091
1000205    1094
1000206     562
1000207    1096
1000208    1097
Name: movie_id, Length: 1000209, dtype: int64

ratings에 있는 유니크한 사용자 수

In [10]:
ratings['user_id'].nunique()

6040

In [11]:
ratings.columns

Index(['user_id', 'movie_id', 'counts', 'timestamp'], dtype='object')

In [12]:
movies = movies.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

In [13]:
movies['movie_id']

0          1
1          2
2          3
3          4
4          5
        ... 
3878    3948
3879    3949
3880    3950
3881    3951
3882    3952
Name: movie_id, Length: 3883, dtype: int64

In [14]:
ratings.sort_values('movie_id')

,user_id,movie_id,counts,timestamp
427702,2599,1,4,973796689
1966,18,1,4,978154768
683688,4089,1,5,965428947
596207,3626,1,4,966594018
465902,2873,1,5,972784317
...,...,...,...,...
84701,551,3952,4,976067330
253845,1544,3952,4,974742620
180689,1130,3952,3,975593522
35180,238,3952,4,976760112


가장 인기 있는 영화 30개(인기순)

In [15]:
movies_count = ratings.groupby('movie_id')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

movie_id
2858    3428
260     2991
1196    2990
1210    2883
480     2672
2028    2653
589     2649
2571    2590
1270    2583
593     2578
1580    2538
1198    2514
608     2513
2762    2459
110     2443
2396    2369
1197    2318
527     2304
1617    2288
1265    2278
1097    2269
2628    2250
2997    2241
318     2227
858     2223
356     2194
2716    2181
296     2171
1240    2098
1       2077
Name: user_id, dtype: int64

#### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [16]:
movie_title_to_id =  dict(zip(movies['title'], movies['movie_id']))

In [17]:
movie_title_to_id['toy story (1995)']

1

In [18]:
my_favorite  = ['Sabrina (1954)' , 'Waiting to Exhale (1995)' ,'cutthroat island (1995)' ,'Jumanji (1995)' ,'Toy Story (1995)']
my_favorite  = pd.Series(my_favorite)
my_favorite  = my_favorite.str.lower()
my_favorite  = list(map(lambda my_favorite: movie_title_to_id[my_favorite], my_favorite))
my_favorite

my_playlist  = pd.DataFrame({'user_id': ['zimin']*5, 'movie_id': my_favorite, 'counts':[4]*5})

if not ratings.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings  = ratings.append(my_playlist)         # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
    
#     ratings = pd.concat(ratings, my_playlist)
    
ratings.tail(20)       # 잘 추가되었는지 확인해 봅시다.


,user_id,movie_id,counts,timestamp
1000194,6040,1077,5,964828799.0
1000195,6040,1079,2,956715648.0
1000196,6040,549,4,956704746.0
1000197,6040,2020,3,956715288.0
1000198,6040,2021,3,956716374.0
1000199,6040,2022,5,956716207.0
1000200,6040,2028,5,956704519.0
1000201,6040,1080,4,957717322.0
1000202,6040,1089,4,956704996.0
1000203,6040,1090,3,956715518.0


In [19]:
user_unique = ratings['user_id'].unique()
user_unique

array([1, 2, 3, ..., 6039, 6040, 'zimin'], dtype=object)

In [20]:
# 고유한 유저, movie_id를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_id_unique = ratings['movie_id'].unique()

# 유저, movie_id indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx        = {v:k for k,v in enumerate(user_unique)}
movie_id_to_idx    = {v:k for k,v in enumerate(movie_id_unique)}

In [21]:
movie_id_unique

array([1193,  661,  914, ..., 2845, 3607, 2909])

In [22]:
user_to_idx[1]

0

In [23]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6040이 나와야 합니다. 
# print(movie_id_to_idx[1])
print(movie_title_to_id['sabrina (1954)'])
print(movie_id_to_idx[movie_title_to_id['sabrina (1954)']])

6040
915
645


In [24]:
ratings.tail(10)

,user_id,movie_id,counts,timestamp
1000204,6040,1091,1,956716541.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,zimin,915,4,NaN
1,zimin,4,4,NaN
2,zimin,15,4,NaN
3,zimin,2,4,NaN
4,zimin,1,4,NaN


In [25]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_id_to_idx 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_id_data = ratings['movie_id'].map(movie_id_to_idx.get).dropna()
if len(temp_movie_id_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_id_data
else:
    print('movie_id column indexing Fail!!')
    
ratings.reset_index()    

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6040,645,4,NaN
1,6040,450,4,NaN
2,6040,1850,4,NaN
3,6040,573,4,NaN


#### 4) CSR matrix를 직접 만들어 봅시다.

In [26]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

In [27]:
num_user  = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

In [28]:
ratings = ratings.reset_index()

In [29]:
ratings

,index,user_id,movie_id,counts,timestamp
0,0,0,0,5,978300760.0
1,1,0,1,3,978302109.0
2,2,0,2,3,978301968.0
3,3,0,3,4,978300275.0
4,4,0,4,5,978824291.0
...,...,...,...,...,...
1000209,0,6040,645,4,NaN
1000210,1,6040,450,4,NaN
1000211,2,6040,1850,4,NaN
1000212,3,6040,573,4,NaN


In [30]:
del ratings['index']

In [31]:
ratings

,user_id,movie_id,counts,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
1000209,6040,645,4,NaN
1000210,6040,450,4,NaN
1000211,6040,1850,4,NaN
1000212,6040,573,4,NaN


In [32]:
ratings.user_id

0             0
1             0
2             0
3             0
4             0
           ... 
1000209    6040
1000210    6040
1000211    6040
1000212    6040
1000213    6040
Name: user_id, Length: 1000214, dtype: int64

In [33]:
num_user

6041

In [34]:
ratings.movie_id

0             0
1             1
2             2
3             3
4             4
           ... 
1000209     645
1000210     450
1000211    1850
1000212     573
1000213      40
Name: movie_id, Length: 1000214, dtype: int64

In [35]:
num_movie

3706

In [36]:
ratings['movie_id'].nunique()

3706

In [37]:
ratings.user_id

0             0
1             0
2             0
3             0
4             0
           ... 
1000209    6040
1000210    6040
1000211    6040
1000212    6040
1000213    6040
Name: user_id, Length: 1000214, dtype: int64

In [38]:
ratings.counts

0          5
1          3
2          3
3          4
4          5
          ..
1000209    4
1000210    4
1000211    4
1000212    4
1000213    4
Name: counts, Length: 1000214, dtype: int64

In [39]:
csr_data =   csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6041x3706 sparse matrix of type '<class 'numpy.int64'>'
	with 1000214 stored elements in Compressed Sparse Row format>

#### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [40]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [41]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [42]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3706x6041 sparse matrix of type '<class 'numpy.int64'>'
	with 1000214 stored elements in Compressed Sparse Column format>

In [43]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [44]:
zimin, sabrina = user_to_idx['zimin'], movie_id_to_idx[movie_title_to_id['sabrina (1954)']]
zimin_vector, sabrina_vector = als_model.user_factors[zimin], als_model.item_factors[sabrina]

In [45]:
als_model.user_factors[zimin]

array([-0.51715946, -0.24762626,  0.3747106 ,  0.0819497 ,  0.24092692,
        0.7544373 , -0.16282345, -0.08713497, -0.03065275,  0.32087794,
        0.32571507, -0.25561023,  0.40749204,  0.24762632, -0.2807281 ,
       -0.65648115,  0.29927614,  0.08306719,  0.0869378 , -0.4727874 ,
        0.56596303,  0.1146056 ,  0.3446015 ,  0.6645778 ,  0.3584802 ,
       -0.27236456, -0.3160228 , -0.34593803, -0.0787219 ,  0.03066489,
        0.24592912,  0.02048984,  0.126095  ,  0.19001126, -0.28840685,
       -0.46237257,  0.03316931, -0.6005802 ,  0.22283913,  0.5193777 ,
       -0.12522696, -0.5094703 ,  0.3433442 , -0.14453626, -0.18188511,
        0.06980613, -0.02567998, -0.53100437, -0.1193027 , -0.287075  ,
       -0.21287015,  0.2591871 , -0.0517579 ,  0.40927485,  0.21852416,
       -0.25966582,  0.30529892,  0.22585095, -0.14122446, -0.10046311,
        0.07176586,  0.00579611,  0.08017123,  0.40582314,  0.08059112,
        0.2657594 ,  0.32507646, -0.12288641,  0.29998416, -0.27

In [46]:
zimin_vector

array([-0.51715946, -0.24762626,  0.3747106 ,  0.0819497 ,  0.24092692,
        0.7544373 , -0.16282345, -0.08713497, -0.03065275,  0.32087794,
        0.32571507, -0.25561023,  0.40749204,  0.24762632, -0.2807281 ,
       -0.65648115,  0.29927614,  0.08306719,  0.0869378 , -0.4727874 ,
        0.56596303,  0.1146056 ,  0.3446015 ,  0.6645778 ,  0.3584802 ,
       -0.27236456, -0.3160228 , -0.34593803, -0.0787219 ,  0.03066489,
        0.24592912,  0.02048984,  0.126095  ,  0.19001126, -0.28840685,
       -0.46237257,  0.03316931, -0.6005802 ,  0.22283913,  0.5193777 ,
       -0.12522696, -0.5094703 ,  0.3433442 , -0.14453626, -0.18188511,
        0.06980613, -0.02567998, -0.53100437, -0.1193027 , -0.287075  ,
       -0.21287015,  0.2591871 , -0.0517579 ,  0.40927485,  0.21852416,
       -0.25966582,  0.30529892,  0.22585095, -0.14122446, -0.10046311,
        0.07176586,  0.00579611,  0.08017123,  0.40582314,  0.08059112,
        0.2657594 ,  0.32507646, -0.12288641,  0.29998416, -0.27

In [47]:
sabrina_vector

array([ 0.00459346, -0.00755392,  0.00682628, -0.00632222,  0.00589759,
        0.01235236,  0.01256577,  0.01652021,  0.02365196,  0.02172532,
        0.01125556, -0.00109858,  0.0098776 ,  0.01671609,  0.00870873,
       -0.00502597,  0.00457401,  0.00818648, -0.00718272, -0.00620245,
        0.01557232,  0.00991896, -0.00612196, -0.00149567, -0.00397076,
        0.01589192,  0.02040808, -0.00916238,  0.00805778, -0.00281231,
        0.01359624, -0.00781282, -0.00618514, -0.00497466,  0.01981206,
        0.00645103,  0.00816534, -0.01610483,  0.00268725,  0.00379285,
        0.00024871,  0.00675448,  0.00217461,  0.00497838, -0.02027326,
        0.00061117,  0.0080498 , -0.00178127,  0.01370111,  0.0148245 ,
       -0.00497884,  0.0105275 , -0.02372598, -0.00042458,  0.00973478,
        0.01868224,  0.00337359, -0.00089508,  0.00509249,  0.02908048,
       -0.01421601,  0.00890952,  0.00633349,  0.00373792, -0.00454124,
        0.00478249, -0.00629784,  0.02092574,  0.01696524,  0.00

In [48]:
# zimin과 sabrina_vector를 내적하는 코드
np.dot(zimin_vector, sabrina_vector)

0.11626412

#### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

선호하는 5가지 영화 중 하나. 훈련된 모델이 예측한 나의 선호도

In [49]:
zimin, toy_story = user_to_idx['zimin'], movie_id_to_idx[movie_title_to_id['toy story (1995)']]
zimin_vector, toy_story_vector = als_model.user_factors[zimin], als_model.item_factors[toy_story]
np.dot(zimin_vector, toy_story_vector)


0.39778548

 그 외의 영화 하나. 훈련된 모델이 예측한 나의 선호도

In [50]:
zimin, father_of_the_bride = user_to_idx['zimin'], movie_id_to_idx[movie_title_to_id['father of the bride part ii (1995)']]
zimin_vector, father_of_the_bride_vector = als_model.user_factors[zimin], als_model.item_factors[father_of_the_bride]
np.dot(zimin_vector, father_of_the_bride_vector)

0.0138608655

#### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [51]:
similar_movies = als_model.similar_items(sabrina, N=15)
similar_movies

[(645, 0.9999999),
 (648, 0.8155113),
 (708, 0.71026844),
 (1503, 0.68105763),
 (1260, 0.6686158),
 (723, 0.6561707),
 (684, 0.60866153),
 (1794, 0.58658075),
 (1778, 0.57262677),
 (82, 0.5654669),
 (1465, 0.5473478),
 (679, 0.54496163),
 (1460, 0.5316141),
 (1451, 0.52181613),
 (2087, 0.49114686)]

In [52]:
#movie_id_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie_id = {v:k for k,v in movie_id_to_idx.items()}
id_to_movie_title = {v:k for k,v in movie_title_to_id.items()}

[id_to_movie_title[idx_to_movie_id[i[0]]] for i in similar_movies]


['sabrina (1954)',
 'roman holiday (1953)',
 'shop around the corner, the (1940)',
 'to catch a thief (1955)',
 'charade (1963)',
 'philadelphia story, the (1940)',
 'ghost and mrs. muir, the (1947)',
 'quiet man, the (1952)',
 'love in the afternoon (1957)',
 "breakfast at tiffany's (1961)",
 'rebecca (1940)',
 'affair to remember, an (1957)',
 'funny face (1957)',
 'notorious (1946)',
 'penny serenade (1941)']

#### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [53]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
movies_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movies_recommended

[(50, 0.42489594),
 (4, 0.23430826),
 (381, 0.21100277),
 (852, 0.16872017),
 (514, 0.16803575),
 (606, 0.16732475),
 (741, 0.16656825),
 (656, 0.15536919),
 (187, 0.1544433),
 (82, 0.1447764),
 (939, 0.14281452),
 (33, 0.13604116),
 (723, 0.12965512),
 (648, 0.12821499),
 (568, 0.12570554),
 (116, 0.124357335),
 (1066, 0.124028906),
 (1037, 0.12218085),
 (673, 0.11913615),
 (678, 0.11835116)]

In [54]:
[id_to_movie_title[idx_to_movie_id[i[0]]] for i in movies_recommended]

['toy story 2 (1999)',
 "bug's life, a (1998)",
 'babe (1995)',
 'indian in the cupboard, the (1995)',
 'mask, the (1994)',
 'santa clause, the (1994)',
 'willy wonka and the chocolate factory (1971)',
 'hook (1991)',
 'dragonheart (1996)',
 "breakfast at tiffany's (1961)",
 'iron giant, the (1999)',
 'aladdin (1992)',
 'philadelphia story, the (1940)',
 'roman holiday (1953)',
 'teenage mutant ninja turtles (1990)',
 'groundhog day (1993)',
 'toys (1992)',
 'legend (1985)',
 'labyrinth (1986)',
 'willow (1988)']

#### 후기  
집에 있는 파이썬이 오동작을 일으켜서 시간이 많이 걸렸다. 다 밀어버릴거다.  
데이터에 있는 인덱스를 사용하려고 해봤는데 오류가 떴다. 인덱스에 인덱스를 메겨서 돌려야 했다.
